# DynamoDB Resource Table Examples

A resource representing an Amazon DynamoDB Table
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html#table

In [1]:
import boto3
dynamodb = boto3.resource('dynamodb')

## Create Table

In [2]:
table = dynamodb.create_table(
    TableName='Movies',
    KeySchema=[
        {
            'AttributeName': 'year',
            'KeyType': 'HASH'  # Partition key
        },
        {
            'AttributeName': 'title',
            'KeyType': 'RANGE'  # Sort key
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'year',
            'AttributeType': 'N'
        },
        {
            'AttributeName': 'title',
            'AttributeType': 'S'
        },

    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 10,
        'WriteCapacityUnits': 10
    }
)

In [3]:
table

dynamodb.Table(name='Movies')

## Put Item

In [4]:
import json
from decimal import Decimal
with open('moviedata.json') as f:
    movie_list=json.load(f,parse_float=Decimal)

In [7]:
table = dynamodb.Table('Movies')
for movie in movie_list[0:5]: # insert 5 items
    print(movie)
    table.put_item(Item=movie)

{'year': 2013, 'title': 'Rush', 'info': {'directors': ['Ron Howard'], 'release_date': '2013-09-02T00:00:00Z', 'rating': Decimal('8.3'), 'genres': ['Action', 'Biography', 'Drama', 'Sport'], 'image_url': 'http://ia.media-imdb.com/images/M/MV5BMTQyMDE0MTY0OV5BMl5BanBnXkFtZTcwMjI2OTI0OQ@@._V1_SX400_.jpg', 'plot': 'A re-creation of the merciless 1970s rivalry between Formula One rivals James Hunt and Niki Lauda.', 'rank': 2, 'running_time_secs': 7380, 'actors': ['Daniel Bruhl', 'Chris Hemsworth', 'Olivia Wilde']}}
{'year': 2013, 'title': 'Prisoners', 'info': {'directors': ['Denis Villeneuve'], 'release_date': '2013-08-30T00:00:00Z', 'rating': Decimal('8.2'), 'genres': ['Crime', 'Drama', 'Thriller'], 'image_url': 'http://ia.media-imdb.com/images/M/MV5BMTg0NTIzMjQ1NV5BMl5BanBnXkFtZTcwNDc3MzM5OQ@@._V1_SX400_.jpg', 'plot': "When Keller Dover's daughter and her friend go missing, he takes matters into his own hands as the police pursue multiple leads and the pressure mounts. But just how far wil

## Get Item

In [9]:
table = dynamodb.Table('Movies')
table.get_item(
    Key={'year':2013,'title':'Rush'},
    ConsistentRead=True,
    ReturnConsumedCapacity='TOTAL'
)

{'Item': {'year': Decimal('2013'),
  'info': {'actors': ['Daniel Bruhl', 'Chris Hemsworth', 'Olivia Wilde'],
   'release_date': '2013-09-02T00:00:00Z',
   'plot': 'A re-creation of the merciless 1970s rivalry between Formula One rivals James Hunt and Niki Lauda.',
   'genres': ['Action', 'Biography', 'Drama', 'Sport'],
   'image_url': 'http://ia.media-imdb.com/images/M/MV5BMTQyMDE0MTY0OV5BMl5BanBnXkFtZTcwMjI2OTI0OQ@@._V1_SX400_.jpg',
   'directors': ['Ron Howard'],
   'rating': Decimal('8.3'),
   'rank': Decimal('2'),
   'running_time_secs': Decimal('7380')},
  'title': 'Rush'},
 'ConsumedCapacity': {'TableName': 'Movies', 'CapacityUnits': 1.0},
 'ResponseMetadata': {'RequestId': 'VI19RHQK11IO3JDE41H8AL2FGBVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 23 Aug 2020 20:50:42 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '675',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'VI19RHQK11IO3JDE

## Update Item

In [15]:
table = dynamodb.Table('Movies')
from decimal import Decimal
table.update_item(
    Key={'year':2013,'title':'Rush'},
    UpdateExpression='set info.rating=:r, info.plot=:p, info.actors=:a',
    ConditionExpression="size(info.actors) > :num",
    ExpressionAttributeValues={
        ':r':Decimal(5.5),
        ':p':'Everything happens all at once',
        ':a':['Larry','Kevin'],
        ':num': 1
    },
    ReturnValues="UPDATED_NEW"
)

{'Attributes': {'info': {'actors': ['Larry', 'Kevin'],
   'plot': 'Everything happens all at once',
   'rating': Decimal('5.5')}},
 'ResponseMetadata': {'RequestId': 'I0HVKJ7BC5SH9U4C563MHKTC1FVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 23 Aug 2020 21:04:34 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '143',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'I0HVKJ7BC5SH9U4C563MHKTC1FVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '657691679'},
  'RetryAttempts': 0}}

## Delete Item

In [16]:
table = dynamodb.Table('Movies')
response = table.delete_item(
    Key={
        'year': 2013,
        'title': 'Rush'
    },
    ConditionExpression="info.rating <= :val",
    ExpressionAttributeValues={
        ":val": Decimal(8)
    }
)

In [17]:
response

{'ResponseMetadata': {'RequestId': 'R1GKCC1TC7SN2CD3TS1LOST9A7VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 23 Aug 2020 21:07:36 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'R1GKCC1TC7SN2CD3TS1LOST9A7VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

## Query

In [19]:
from boto3.dynamodb.conditions import Key
table = dynamodb.Table('Movies')
table.query(
    KeyConditionExpression=Key('year').eq(2013) & Key('title').eq('This Is the End')
)

{'Items': [{'year': Decimal('2013'),
   'info': {'actors': ['James Franco', 'Jonah Hill', 'Seth Rogen'],
    'release_date': '2013-06-03T00:00:00Z',
    'plot': "While attending a party at James Franco's house, Seth Rogen, Jay Baruchel and many other celebrities are faced with the apocalypse.",
    'genres': ['Comedy', 'Fantasy'],
    'image_url': 'http://ia.media-imdb.com/images/M/MV5BMTQxODE3NjM1Ml5BMl5BanBnXkFtZTcwMzkzNjc4OA@@._V1_SX400_.jpg',
    'directors': ['Evan Goldberg', 'Seth Rogen'],
    'rating': Decimal('7.2'),
    'rank': Decimal('6'),
    'running_time_secs': Decimal('6420')},
   'title': 'This Is the End'}],
 'Count': 1,
 'ScannedCount': 1,
 'ResponseMetadata': {'RequestId': 'PB70PMR775H26AHS6DDP9M534NVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 23 Aug 2020 21:14:44 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '672',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'PB70

## Scan

In [23]:
table = dynamodb.Table('Movies')
scan_kwargs = {
    'FilterExpression': Key('year').between(1960,2020),
    'ProjectionExpression': "#yr, title, info.rating",
    'ExpressionAttributeNames': {"#yr": "year"}
}

done = False
start_key = None
while not done:
    if start_key:
        scan_kwargs['ExclusiveStartKey'] = start_key
    response = table.scan(**scan_kwargs)
    print(response)
    start_key = response.get('LastEvaluatedKey', None)
    done = start_key is None

{'Items': [{'info': {'rating': Decimal('8.2')}, 'year': Decimal('2013'), 'title': 'Prisoners'}, {'year': Decimal('2013'), 'title': 'The Hunger Games: Catching Fire'}, {'info': {'rating': Decimal('7.2')}, 'year': Decimal('2013'), 'title': 'This Is the End'}, {'year': Decimal('2013'), 'title': 'Thor: The Dark World'}], 'Count': 4, 'ScannedCount': 4, 'ResponseMetadata': {'RequestId': '1OOUF87JR72DTF0MK0LQ1D7HOVVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Sun, 23 Aug 2020 21:22:08 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '341', 'connection': 'keep-alive', 'x-amzn-requestid': '1OOUF87JR72DTF0MK0LQ1D7HOVVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '212092884'}, 'RetryAttempts': 0}}


## Delete Table

In [24]:
table = dynamodb.Table('Movies')
table.delete()

{'TableDescription': {'TableName': 'Movies',
  'TableStatus': 'DELETING',
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 10,
   'WriteCapacityUnits': 10},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:us-east-1:226595923650:table/Movies',
  'TableId': '19ce7c0d-2d60-4893-b6fe-daf55215393d'},
 'ResponseMetadata': {'RequestId': 'FUIO3JARGQRAMLESHTI64JQLGNVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 23 Aug 2020 21:23:09 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '316',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'FUIO3JARGQRAMLESHTI64JQLGNVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '1843616595'},
  'RetryAttempts': 0}}